In [1]:
# Dependencies
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey


In [2]:
# load CSV
hawaii_stations = 'Resources/clean_hawaii_stations.csv'
hawaii_measurements = 'Resources/clean_hawaii_measurements.csv'

# Read with pandas
stations_df = pd.read_csv(hawaii_stations)
measurements_df = pd.read_csv(hawaii_measurements)

measurements_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [3]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///hawaii.sqlite')

# Create a connection to the engine called `conn`
conn = engine.connect()

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
# Create the Station and Measurement classes

# Station table
class Station(Base):
    __tablename__ = 'station'

    station = Column(String(12), primary_key=True)
    name = Column(String(100))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
# Measurement table
class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String(12), ForeignKey('station.station'))
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)

In [5]:
# Create the tables within the database
Base.metadata.create_all(engine)

In [6]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
stations_data = stations_df.to_dict(orient='records')
stations_data[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [7]:
measurements_data = measurements_df.to_dict(orient='records')
measurements_data[0]

{'Unnamed: 0': 0,
 'date': '2010-01-01',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [8]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [9]:
# Save the reference to the tables as a variable.

stations_table = sqlalchemy.Table('station', metadata, autoload=True)
measurements_table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [10]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.

#TAs please skip this step if running for the first time
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [11]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurements_table.insert(), measurements_data)
conn.execute(stations_table.insert(), stations_data)

In [12]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 20").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (6, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (7, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (8, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (9, 'USC00519397', '2010-01-11', 0.01, 64.0),
 (10, 'USC00519397', '2010-01-12', 0.0, 61.0),
 (11, 'USC00519397', '2010-01-14', 0.0, 66.0),
 (12, 'USC00519397', '2010-01-15', 0.0, 65.0),
 (13, 'USC00519397', '2010-01-16', 0.0, 68.0),
 (14, 'USC00519397', '2010-01-17', 0.0, 64.0),
 (15, 'USC00519397', '2010-01-18', 0.0, 72.0),
 (16, 'USC00519397', '2010-01-19', 0.0, 66.0),
 (17, 'USC00519397', '2010-01-20', 0.0, 66.0),
 (18, 'USC00519397', '2010-01-21', 0.0, 69.0),
 (19, 'USC00519397', '2010-01-22', 0.0, 67.0),
 (20, 'USC00519397', '2010-01-23', 0.0, 67.0)]